In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

In [2]:
import openpyxl

In [3]:
from selenium import webdriver
import time
import datetime

### 새로운 워크북 파일 생성

Keyword에 학교폭력 "학교폭력"을 입력받은 후 엑셀 파일을 만듬.

Word Cloud 만들 때 본문만 필요하므로 기사의 본문만 수집.

In [44]:
keyword = input("검색어를 입력해주세요: ")
#학교폭력 "학교폭력"

try:
    wb = openpyxl.load_workbook("2021_학교폭력_뉴스.xlsx") #존재하는 파일에다가 새로운 데이터 누적하여 입력
    sheet = wb.active
    print("불러오기 완료")
except:
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["본문"])
    print("새로운 파일을 만들었습니다")

검색어를 입력해주세요: 학교폭력 "학교폭력"
불러오기 완료


### Chrome 웹드라이버 실행

In [45]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https://news.naver.com/')

In [46]:
# 검색창 : input.text_index
# 검색버튼 : button.btn_search_lnb
time.sleep(1)
search = driver.find_element_by_css_selector('input.text_index')
search.send_keys(keyword)
button = driver.find_element_by_css_selector('button.btn_search_lnb')
button.click()

In [47]:
driver.switch_to.window(driver.window_handles[1])
print("Current Page Title is : %s" %driver.title)

Current Page Title is : 학교폭력 "학교폭력" : 네이버 뉴스검색


### 기간 설정

In [48]:
# 검색옵션 : a#search_option_button.bt_option
# 날짜 : div._search_option_period_
# 기간(이전) : input#news_input_period_begin.input_txt._input_text_
# 기간(이후) : input#news_input_period_end.input_txt._input_text_
# 적용 : div._search_option_period_ button._btn_submit_

# 검색옵션 열기
time.sleep(3)
search_opt_box = driver.find_element_by_xpath('//*[@id="search_option_button"]')
search_opt_box.click()
time.sleep(1)

tablist_box = driver.find_element_by_xpath('//div[@class="snb_inner"]/ul[@role="tablist" and @class="option_menu"]')

tablist_elem_list = tablist_box.find_elements_by_xpath('./li[@role="presentation"]')
press_box = [t for t in tablist_elem_list if t.text == '기간'][0].find_element_by_xpath('./a')
press_box.click()

In [49]:
date_frame = driver.find_element_by_xpath('.//div[@class="snb_itembox type_news _search_option_period_"]')

start_dt = datetime.datetime.strptime('2021.01.01', '%Y.%m.%d')
last_dt = datetime.datetime.strptime('2021.02.28', '%Y.%m.%d')
ds = start_dt.strftime('%Y.%m.%d')
de = last_dt.strftime('%Y.%m.%d')
min_date = driver.find_element_by_css_selector('input#news_input_period_begin.input_txt._input_text_')
min_date.send_keys(ds)
max_date = driver.find_element_by_css_selector('input#news_input_period_end.input_txt._input_text_')
max_date.send_keys(de)
button = driver.find_element_by_css_selector('div._search_option_period_ button._btn_submit_')
button.click()

### 언론사 설정

In [50]:
press_list = ['연합뉴스','JTBC']
tablist_box = driver.find_element_by_xpath('//div[@class="snb_inner"]/ul[@role="tablist" and @class="option_menu"]')
tablist_elem_list = tablist_box.find_elements_by_xpath('./li[@role="presentation"]')
press_box = [t for t in tablist_elem_list if t.text == '언론사'][0].find_element_by_xpath('./a')
press_box.click()

activated_press_frame = driver.find_element_by_css_selector('div.snb_itembox.lst_press')
total_press_box = activated_press_frame.find_element_by_css_selector('div#order_cat.group_sort')
press_cat_active_button = total_press_box.find_elements_by_xpath('.//a[@role="tab" and @class="item _tab_filter_"]')
press_cat_active_button_dict = dict(zip([t.text for t in press_cat_active_button], press_cat_active_button))

each_press_box_list = total_press_box.find_elements_by_xpath('.//div[@class="scroll_area _panel_filter_"]')

for idx, press_cat_name in enumerate(press_cat_active_button_dict.keys()):
    #하나의 언론사 종류를 클릭해서 활성화시킴
    press_cat_active_button_dict[press_cat_name].click()
    time.sleep(0.5)
    
    # 선택한 언론사 종류 안의 개별 언론사가 담긴 박스
    each_press_box = each_press_box_list[idx].find_element_by_xpath('./div[@class="select_item"]')
    # 개별 언론사의 이름
    each_press_title_list = [ep.get_attribute('title') for ep in each_press_box.find_elements_by_xpath('.//label')]
    # 개별 언론사 체크 박스
    each_press_input_list = each_press_box.find_elements_by_xpath('.//input')
    

    # 딕셔너리(개별 언론사 이름 : 개별 언론사 체크 박스)
    each_press_title_input_dict = dict(zip(each_press_title_list, each_press_input_list))
    # 추출하고 싶은 언론사 존재 시 체크박스 클릭
    for title in [tit for tit in each_press_title_input_dict.keys() if tit in press_list]:
        print(title)
        each_press_title_input_dict[title].click()
        
button = driver.find_element_by_css_selector('button.type_default._btn_submit_')
button.click()

연합뉴스
JTBC


### 기사 수집

In [51]:
while True:
    time.sleep(1)
    articles = driver.find_elements_by_css_selector('div.news_area')
    time.sleep(1)
    if len(articles) == 10:
        for article in articles:
            #title = article.find_element_by_css_selector('a.news_tit').text
            #dt = article.find_element_by_css_selector('span.info').text
            driver.switch_to.window(driver.window_handles[1])
            url = article.find_element_by_css_selector('a.news_tit')
            url = url.get_attribute('href')
            
            try:

                #open new tab
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[2])
                driver.get(url)

                time.sleep(1)

                ## 기사본문: div#newsEndContents  //  div#articleBodyContents
                try:
                    content = driver.find_element_by_css_selector('div.article_content').text #JTBC
                except:
                    try:
                        content = driver.find_element_by_css_selector('div.article').text #연합뉴스
                    except:
                        content = ' '
                        pass
                sheet.append([ content])
                # close new tab
                driver.close()

                try:
                    driver.switch_to.window(driver.window_handles[2])
                    driver.close()
                except:
                    pass

                time.sleep(1)
                # naver news tab
                driver.switch_to.window(driver.window_handles[1])
            
            except:
                try:
                    driver.switch_to.window(driver.window_handles[2])
                    driver.close()
                except:
                    pass
            
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[1])
                    
        
        page_bar = driver.find_elements_by_css_selector('div.sc_page > *')
        page_bar[2].click()
            
    else:
        for article in articles:
            driver.switch_to.window(driver.window_handles[1])
            url = article.find_element_by_css_selector('a.news_tit')
            url = url.get_attribute('href')

            #open new tab
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[2])
            driver.get(url)

            time.sleep(1)


            try:
                content = driver.find_element_by_css_selector('div.article_content').text #JTBC
            except:
                try:
                    content = driver.find_element_by_css_selector('div.article').text #연합뉴스
                except:
                    content = ' '
                    pass
            sheet.append([content])
            # close new tab
            driver.close()
            
            try:
                driver.switch_to.window(driver.window_handles[2])
                driver.close()
            except:
                pass
            
            time.sleep(1)
            # naver news tab
            driver.switch_to.window(driver.window_handles[1])
        break

### 엑셀 파일로 저장

In [52]:
wb.save("2021_학교폭력_뉴스.xlsx")
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()

In [53]:
import pandas as pd

data = pd.read_excel('2021_학교폭력_뉴스.xlsx')
print(data.shape)
data.head()

(334, 1)


,본문
0,- 코로나19로 면대면 학폭 줄었으나 와이파이셔틀 등 사이버 학폭 늘어\n- '학교...
1,<정지은 전 트라이애슬론 선수>\n- 故 최숙현 사건·팀 내 폭행 증언 후 재계약 ...
2,김승욱 기자\n기자 페이지\n화장실·기숙사 등 불법카메라 점검…디지털성범죄 예방교육...
3,김동철 기자\n기자 페이지\n전북교육청\n[연합뉴스 자료사진]\n(전주=연합뉴스) ...
4,"""야, 카톡 바로 안 보냐? 죽고 싶어서 환장을 했지? 두고 봐.""\n\n이 앱을 ..."
